# Importaciones y configuración básica

In [8]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

tf.random.set_seed(7)
np.random.seed(7)

SEQ_LEN = 20          # longitud de contexto en palabras (antes: contexto en chars)
EMBED_DIM = 128
RNN_UNITS = 256
BATCH_SIZE = 64
EPOCHS = 20


### Cargar texto crudo

In [4]:
with open('datasets/texto.txt', 'r', encoding='utf-8', errors='ignore') as f:
    raw_text = f.read()

print(raw_text[:1000])  # vistazo

Prologo
En 1953, Isaac Asimov publico Segunda Fundación, el tercer libro de la saga de la Fundación (o el decimotercero según otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su primera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccion que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redacción, Arkady esta utilizando un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo, que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Processor Units), que solo cuestan alrededor de 100 euros, que estarían en la lista del Top500 hac

## Limpieza ligera y normalización (opcional, pero útil para vocabulario)

In [5]:
import re

def normalize_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r'\s+', ' ', s)              # colapsa espacios
    s = s.replace('“','"').replace('”','"').replace('’',"'")
    s = s.replace('—','-').replace('–','-')
    return s.strip()

text = normalize_text(raw_text)
print(text[:300])


prologo en 1953, isaac asimov publico segunda fundación, el tercer libro de la saga de la fundación (o el decimotercero según otras fuentes, este es un tema de debate). en segunda fundacion aparece por primera vez arkady darell, uno de los principales personajes de la parte final de la saga. en su p


### Tokenización a palabras y vectorización

In [6]:
from tensorflow.keras.layers import TextVectorization

MAX_VOCAB = None

vectorizer = TextVectorization(
    standardize=None,          
    split='whitespace',
    max_tokens=MAX_VOCAB,
    output_mode='int',
    output_sequence_length=None  
)

text_ds = tf.data.Dataset.from_tensor_slices([text])
vectorizer.adapt(text_ds)

vocab = vectorizer.get_vocabulary()              
word2id = {w:i for i, w in enumerate(vocab)}     

tokens = vectorizer(tf.constant([text])).numpy()[0]  
vocab_size = len(vocab)
vocab_size, tokens[:20]


(5900,
 array([3030,    4, 2247, 1800, 2147, 2998,  110, 4104,    6, 1072,   63,
           2,    5, 2784,    2,    5, 4105,  355,    6, 4687]))

### Construcción del dataset (ventanas de N palabras → siguiente palabra)

In [9]:
def build_windows(token_ids: np.ndarray, seq_len: int):
    total = len(token_ids) - seq_len
    X = np.zeros((total, seq_len), dtype=np.int32)
    y = np.zeros((total,), dtype=np.int32)
    for i in range(total):
        X[i] = token_ids[i: i+seq_len]
        y[i] = token_ids[i+seq_len]
    return X, y

X, y = build_windows(tokens, SEQ_LEN)
len(tokens), X.shape, y.shape


(31260, (31240, 20), (31240,))

In [10]:
ds = tf.data.Dataset.from_tensor_slices((X, y))
ds = ds.shuffle(buffer_size=min(10000, len(X))).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 20), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [11]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=EMBED_DIM, input_length=SEQ_LEN),
    GRU(RNN_UNITS, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',   
    metrics=['accuracy']
)

model.summary()


c:\Users\juanp\Programacion\Redes-Neuronales-Notebook\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Entrenamiento con checkpoint

In [12]:
os.makedirs('checkpoints_word', exist_ok=True)
ckpt = ModelCheckpoint(
    filepath='checkpoints_word/weights.{epoch:02d}-{loss:.3f}.keras',
    save_weights_only=False,
    save_best_only=False
)

history = model.fit(ds, epochs=EPOCHS, callbacks=[ckpt])

Epoch 1/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 27s 47ms/step - accuracy: 0.0990 - loss: 6.7370
Epoch 2/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.1364 - loss: 5.7660
Epoch 3/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.1702 - loss: 5.2071
Epoch 4/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.2025 - loss: 4.6346
Epoch 5/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.2395 - loss: 4.0317
Epoch 6/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 23s 47ms/step - accuracy: 0.2959 - loss: 3.4182
Epoch 7/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.3934 - loss: 2.8152
Epoch 8/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.4945 - loss: 2.2817
Epoch 9/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.5889 - loss: 1.8454
Epoch 10/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 45ms/step - accuracy: 0.6711 - loss: 1.4939
Epoch 11/20
489/489 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - accuracy: 0.7372 - loss: 1.2087
Epoch 12/20
489/489 ━━━━━━━━━━

### Función de muestreo (temperatura) y helpers

In [13]:
def sample_from_logits(probs, temperature: float = 1.0):
    """Muestrea un índice de palabra a partir de una distribución (probs) ajustada por temperatura."""
    probs = np.asarray(probs).astype(np.float64)
    if temperature <= 0:
        return int(np.argmax(probs))
    logits = np.log(probs + 1e-8) / temperature
    exp = np.exp(logits)
    adjusted = exp / np.sum(exp)
    return int(np.random.choice(len(adjusted), p=adjusted))

id2word = vocab

def ids_to_text(ids):
    return ' '.join(id2word[i] for i in ids if i < len(id2word) and i > 0)


### Generador de texto palabra-por-palabra

In [14]:
def generate_text(model, seed_text: str, num_words: int = 50, temperature: float = 1.0):
    seed_norm = normalize_text(seed_text)
    seed_ids = vectorizer(tf.constant([seed_norm])).numpy()[0].tolist()
    seed_ids = [i for i in seed_ids if i != 0]

    context = seed_ids[-SEQ_LEN:]
    if len(context) < SEQ_LEN:
        context = [0]*(SEQ_LEN - len(context)) + context

    generated = []

    for _ in range(num_words):
        x = np.array([context[-SEQ_LEN:]], dtype=np.int32)
        probs = model.predict(x, verbose=0)[0]
        next_id = sample_from_logits(probs, temperature=temperature)
        generated.append(next_id)
        context.append(next_id)

    return ids_to_text(seed_ids + generated)



In [15]:
for temp in [0.0, 0.7, 1.0, 1.3]:
    print(f"\n--- temperature = {temp} ---")
    print(generate_text(model, seed_text="en aquel tiempo", num_words=40, temperature=temp))


--- temperature = 0.0 ---
en aquel tiempo y muchos otros nvidia nvidia v100 from keras.layers import to_categorical (train_images, train_labels), (test_images, test_labels) = mnist.load_data() train_images = np.random.rand(n) / = 1s 14us/step - loss: 1.5102 - acc: 0.6537 epoch 5/5 60000/60000 [==================] - 1s 13us/step - loss: 1.6978 -

--- temperature = 0.7 ---
en aquel tiempo y error: traduccion de infraestructuras altamente paralelas. pytorch y la version permite que las mi seguidas a todos los datos de entrenar redes del mundo de programacion general, en esta pagina al escribir las vision por computador. y mas de

--- temperature = 1.0 ---
en aquel tiempo o data cara. pero si tenemos las gpus, el eje llamado crear la tareas de un tensor negativo, que veremos en mas de dos clases. por eso usaremos lo llamaremos explicado en el capitulo 3 un capitulo definido por lo

--- temperature = 1.3 ---
en aquel tiempo por dia, son cruciales para considerar mi mayo teoricos donde pres